In [1]:
from flask import Flask, render_template, Response
import cv2
import os

app = Flask(__name__)

# Path to Haar Cascade model for number plate detection
harcascade = "number.xml"
plate_cascade = cv2.CascadeClassifier(harcascade)  # Load Haar cascade model
min_area = 500  # Minimum area for a detected plate

# Initialize Video Capture
cap = cv2.VideoCapture("video.mp4")

count_file = "plates/count.txt"
if os.path.exists(count_file):
    with open(count_file, "r") as f:
        content = f.read().strip()
        count = int(content) if content.isdigit() else 0
else:
    count = 0


def detect_number_plate():
    global count
    while True:
        success, img = cap.read()
        if not success:
            break  # Stop when video ends

        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        plates = plate_cascade.detectMultiScale(img_gray, 1.1, 4)

        for x, y, w, h in plates:
            area = w * h
            if area > min_area:
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(
                    img,
                    "Number Plate",
                    (x, y - 5),
                    cv2.FONT_HERSHEY_COMPLEX_SMALL,
                    1,
                    (255, 0, 255),
                    2,
                )

                img_roi = img[y : y + h, x : x + w]

                # Save number plate when detected
                filename = f"plates/scanned_img_{count}.jpg"
                cv2.imwrite(filename, img_roi)

                count += 1
                with open(count_file, "w") as f:
                    f.write(str(count))

        # Encode the frame for streaming
        ret, buffer = cv2.imencode(".jpg", img)
        frame = buffer.tobytes()

        yield (b"--frame\r\n" b"Content-Type: image/jpeg\r\n\r\n" + frame + b"\r\n")


@app.route("/")
def index():
    return render_template("index.html")


@app.route("/video_feed")
def video_feed():
    return Response(
        detect_number_plate(), mimetype="multipart/x-mixed-replace; boundary=frame"
    )


if __name__ == "_main_":
    app.run(debug=True)
